In [ ]:
import pdal
import json
import numpy as np

# Input and output file paths
input_las = "/mnt/floorheightvolume/lidar_Wagga/clipped/gnaf_GANSW705562005_UFI_957.las"
output_las = "test_ground_points.las"

# Create a PDAL pipeline to extract ground points (classification == 2)
pipeline_json = {
    "pipeline": [
        {
            "type": "readers.las",
            "filename": input_las
        },
        {
            "type": "filters.range",
            "limits": "Classification[2:2]"  # Ground classification is typically 2
        }
    ]
}

# Create and execute the pipeline
pipeline = pdal.Pipeline(json.dumps(pipeline_json))
pipeline.execute()

# Get the ground points as a numpy array
ground_points = pipeline.arrays[0]

# Extract Z values (elevations)
elevations = ground_points['Z']

# Calculate statistics
stats = {
    'count': len(elevations),
    'mean': np.mean(elevations),
    'median': np.median(elevations),
    'min': np.min(elevations),
    'max': np.max(elevations),
    'std_dev': np.std(elevations),
    'percentile_25': np.percentile(elevations, 25),
    'percentile_75': np.percentile(elevations, 75)
}

# Print the statistics
print("Ground Elevation Statistics:")
for key, value in stats.items():
    print(f"{key:>12}: {value:.3f}")

# Optional: Print basic stats in a single line
print(f"\nSummary: Count={stats['count']}, Mean={stats['mean']:.2f}, "
      f"Min={stats['min']:.2f}, Max={stats['max']:.2f}, "
      f"StdDev={stats['std_dev']:.2f}")

In [ ]:
import matplotlib.pyplot as plt

plt.hist(elevations, bins=50)
plt.title("Ground Elevation Distribution")
plt.xlabel("Elevation")
plt.ylabel("Count")
plt.show()

In [ ]:
# Assuming you have X and Y coordinates
x = ground_points['X']
y = ground_points['Y']
z = ground_points['Z']

# Create bins (example: 100m grid cells)
x_bins = np.arange(np.min(x), np.max(x), 0.5)
y_bins = np.arange(np.min(y), np.max(y), 0.5)

# Calculate mean elevation per grid cell
from scipy.stats import binned_statistic_2d
# Calculate statistics
mean_elev, x_edges, y_edges, _ = binned_statistic_2d(
    x, y, z, 
    statistic='mean', 
    bins=[x_bins, y_bins]
)

# Create the plot
plt.figure(figsize=(10, 8))

# Create meshgrid for plotting
x_centers = (x_edges[:-1] + x_edges[1:]) / 2
y_centers = (y_edges[:-1] + y_edges[1:]) / 2
X, Y = np.meshgrid(x_centers, y_centers)

# Plot the data
im = plt.pcolormesh(X, Y, mean_elev.T, shading='auto', cmap='terrain')
plt.colorbar(im, label='Elevation')

# Add point markers where data exists
valid_points = ~np.isnan(mean_elev.T)
plt.scatter(X[valid_points], Y[valid_points], color='black', s=1, alpha=0.3)

plt.title(f'Mean Ground Elevation (Bin Size: {0.5:.1f} units)')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
output_tiff='test_dtm_lidar.tif'
resolution=0.1

# Get CRS from source file first
pipeline_info = {
    "pipeline": [{
        "type": "readers.las",
        "filename": input_las
    }]
}
info_pipeline = pdal.Pipeline(json.dumps(pipeline_info))
info_pipeline.execute()
metadata = info_pipeline.metadata
crs = metadata['metadata']['readers.las']['srs']['horizontal']

# Main processing pipeline
pipeline_json = {
    "pipeline": [
        {
            "type": "readers.las",
            "filename": input_las
        },
        {
            "type": "filters.range",
            "limits": "Classification[2:2]"  # Ground points only
        },
        {
            "type": "writers.gdal",
            "filename": output_tiff,
            "dimension": "Z",  # Export elevation values
            "output_type": "idw",  # Inverse Distance Weighting
            "resolution": resolution,
            "radius": resolution * 3,  # Search radius
            "gdaldriver": "GTiff",
            "data_type": "float32",
            "nodata": -9999,
            "override_srs": crs  # Explicitly set output CRS
        }
    ]
}

pipeline = pdal.Pipeline(json.dumps(pipeline_json))
pipeline.execute()

# Verify output
print(f"Created DTM with CRS: {crs}")
print("Pipeline metadata:", pipeline.metadata)